# Phytoplankton Size Class - Turner

## Summary
Function to calculate phytoplankton size classes using the Northeast U.S. regionally tuned phytoplankton size class algorithm based on Turner et al. (2021)

### References
Turner, K. J., C. B. Mouw, K. J. W. Hyde, R. Morse, and A. B. Ciochetto (2021), Optimization and assessment of phytoplankton size class algorithms for ocean color data on the Northeast U.S. continental shelf, Remote Sensing of Environment, 267, 112729, [doi:https://doi.org/10.1016/j.rse.2021.112729]

In [1]:
import xarray as xr
import pandas
import numpy as np
from datetime import datetime
from getSST8day import SST8day
from getCHL import getCHL

### Read the SST look up file

### Create the PSC function

In [15]:
def psc(chlarr,sstarr):
    sstlut = pandas.read_csv('TURNER_PSIZE_SST_LUT_VER1.csv',index_col='SST')
    sstlut = sstlut.to_xarray()
    
    # ===> Find the coefficients from the LUT based on the input SST
    sst_coeffs = sstlut.sel({"SST": sstarr}, method="nearest")

    # ===> Calculate the phytoplankton size class fractions
    fpico = (sst_coeffs.COEFF3 * (1 - np.exp(-1 * (sst_coeffs.COEFF4 / sst_coeffs.COEFF3) * chlarr))) / chlarr
    fnanopico = (sst_coeffs.COEFF1 * (1 - np.exp(-1 * (sst_coeffs.COEFF2 / sst_coeffs.COEFF1) * chlarr))) / chlarr  
    fnano = fnanopico - fpico
    fmicro = (chlarr - (sst_coeffs.COEFF1 * (1 - np.exp(-1 * (sst_coeffs.COEFF2 / sst_coeffs.COEFF1) * chlarr)))) / chlarr 

    phyto = xr.Dataset({"fpico":fpico,"fnanopico":fnanopico,"fnano":fnano,"fmicro":fmicro,"chlor_a":chlarr})
    
    return phyto

### Test the function

In [18]:
chl = xr.DataArray([0.1,0.3,0.2],dims='x')
sst = xr.DataArray([-10.0,9.0,48.5],dims='x')
psize = psc(chl,sst)
psize

<xarray.Dataset> Size: 144B
Dimensions:    (x: 3)
Coordinates:
    SST        (x) float64 24B 6.777 9.011 21.39
Dimensions without coordinates: x
Data variables:
    fpico      (x) float64 24B 0.2426 0.2646 0.3766
    fnanopico  (x) float64 24B 0.5609 0.6441 0.77
    fnano      (x) float64 24B 0.3183 0.3795 0.3934
    fmicro     (x) float64 24B 0.4391 0.3559 0.23
    chlor_a    (x) float64 24B 0.1 0.3 0.2

### Get the CHL and SST data

In [9]:
chl = getCHL()
chlarr = chl['chlor_a']

QUEUEING TASKS | :   0%|          | 0/2 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/2 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
chlarr["time"]=chlarr["time"].data.astype("datetime64[D]").astype("datetime64[ns]")

In [11]:
chlarr

<xarray.DataArray 'chlor_a' (time: 2, lat: 300, lon: 350)> Size: 840kB
array([[[       nan,        nan,        nan, ..., 0.34472683,
         0.39119378, 0.4430575 ],
        [       nan,        nan,        nan, ..., 0.29670656,
         0.3427215 , 0.39442492],
        [       nan,        nan,        nan, ..., 0.29692715,
         0.29620177, 0.3326351 ],
        ...,
        [       nan,        nan,        nan, ..., 0.05317241,
         0.05361534, 0.06272691],
        [       nan,        nan,        nan, ..., 0.04282099,
         0.05508374, 0.05952638],
        [       nan,        nan,        nan, ..., 0.04031983,
         0.05464929, 0.06157215]],

       [[       nan,        nan,        nan, ..., 0.34244117,
         0.3769699 , 0.39940786],
        [       nan,        nan,        nan, ..., 0.35007438,
         0.34590805, 0.38406527],
        [       nan,        nan,        nan, ..., 0.34907687,
         0.3547749 , 0.34587607],
        ...,
        [       nan,        nan,        nan, ..., 0.04682265,
         0.04544988, 0.04979814],
        [       nan,        nan,        nan, ..., 0.04704475,
         0.04589888, 0.05101591],
        [       nan,        nan,        nan, ..., 0.04556489,
         0.04728106, 0.04951475]]], dtype=float32)
Coordinates:
  * lat      (lat) float32 1kB 49.95 49.85 49.75 49.65 ... 20.25 20.15 20.05
  * lon      (lon) float32 1kB -79.95 -79.85 -79.75 ... -45.25 -45.15 -45.05
  * time     (time) datetime64[ns] 16B 2024-07-19 2024-07-27

In [6]:
sstarr = SST8day('2024-7-19',end_date='2024-8-3')

In [12]:
sstarr

<xarray.DataArray 'sea_surface_temperature' (time: 2, latitude: 1500,
                                             longitude: 1750)> Size: 21MB
array([[[       nan,        nan,        nan, ..., 15.434    ,
         15.42     , 15.366666 ],
        [       nan,        nan,        nan, ..., 15.3880005,
         15.381667 , 15.419999 ],
        [       nan,        nan,        nan, ..., 15.324    ,
         15.386666 , 15.429999 ],
        ...,
        [29.710001 , 29.72     , 29.741251 , ..., 27.57125  ,
         27.578571 , 27.612856 ],
        [29.70375  , 29.730001 , 29.742498 , ..., 27.599998 ,
         27.6      , 27.61     ],
        [29.7125   , 29.74     , 29.745003 , ..., 27.644287 ,
         27.61143  , 27.588572 ]],

       [[       nan,        nan,        nan, ..., 17.312    ,
         17.354    , 17.46     ],
        [       nan,        nan,        nan, ..., 17.183332 ,
         17.492    , 17.513998 ],
        [       nan,        nan,        nan, ..., 17.101667 ,
         17.291666 , 17.4275   ],
        ...,
        [29.735    , 29.710001 , 29.700003 , ..., 27.552856 ,
         27.577145 , 27.587143 ],
        [29.735    , 29.716667 , 29.715002 , ..., 27.560001 ,
         27.547146 , 27.570002 ],
        [29.765001 , 29.761667 , 29.738333 , ..., 27.558573 ,
         27.58     , 27.562857 ]]], dtype=float32)
Coordinates:
  * latitude   (latitude) float32 6kB 49.99 49.97 49.95 ... 20.05 20.03 20.01
  * longitude  (longitude) float32 7kB -79.99 -79.97 -79.95 ... -45.03 -45.01
  * time       (time) datetime64[ns] 16B 2024-07-19 2024-07-27
Attributes:
    colorBarMaximum:        35.0
    colorBarMinimum:        0.0
    comment:                SST obtained by regression with buoy measurements...
    coverage_content_type:  physicalMeasurement
    ioos_category:          Temperature
    long_name:              sea surface sub-skin temperature
    source:                 NOAA
    standard_name:          sea_surface_subskin_temperature
    units:                  degree_C
    valid_max:              40.0
    valid_min:              -2.0

### Regrid the SST to match the CHL

In [13]:
sstregrid = sstarr.interp(latitude=chlarr["lat"],longitude=chlarr["lon"],method='nearest')
sstregrid

<xarray.DataArray 'sea_surface_temperature' (time: 2, lat: 300, lon: 350)> Size: 840kB
array([[[      nan,       nan,       nan, ..., 14.729999, 15.276001,
         15.324   ],
        [      nan,       nan,       nan, ..., 13.902   , 14.606   ,
         15.256   ],
        [      nan,       nan,       nan, ..., 13.851999, 13.894   ,
         14.466001],
        ...,
        [29.649998, 29.694286, 29.715715, ..., 27.625713, 27.637142,
         27.637142],
        [29.6825  , 29.708572, 29.777143, ..., 27.60857 , 27.625717,
         27.63    ],
        [29.741251, 29.712858, 29.78143 , ..., 27.720001, 27.630001,
         27.57125 ]],

       [[      nan,       nan,       nan, ..., 16.886665, 16.706   ,
         17.101667],
        [      nan,       nan,       nan, ..., 16.406002, 16.816   ,
         16.832   ],
        [      nan,       nan,       nan, ..., 15.858   , 16.028   ,
         16.395   ],
        ...,
        [29.713331, 29.794   , 29.748001, ..., 27.843748, 27.692858,
         27.56286 ],
        [29.720001, 29.75    , 29.835714, ..., 27.830002, 27.77857 ,
         27.582   ],
        [29.700003, 29.713333, 29.807142, ..., 27.757143, 27.57429 ,
         27.552856]]], dtype=float32)
Coordinates:
  * time       (time) datetime64[ns] 16B 2024-07-19 2024-07-27
    latitude   (lat) float32 1kB 49.95 49.85 49.75 49.65 ... 20.25 20.15 20.05
    longitude  (lon) float32 1kB -79.95 -79.85 -79.75 ... -45.25 -45.15 -45.05
  * lat        (lat) float32 1kB 49.95 49.85 49.75 49.65 ... 20.25 20.15 20.05
  * lon        (lon) float32 1kB -79.95 -79.85 -79.75 ... -45.25 -45.15 -45.05
Attributes:
    colorBarMaximum:        35.0
    colorBarMinimum:        0.0
    comment:                SST obtained by regression with buoy measurements...
    coverage_content_type:  physicalMeasurement
    ioos_category:          Temperature
    long_name:              sea surface sub-skin temperature
    source:                 NOAA
    standard_name:          sea_surface_subskin_temperature
    units:                  degree_C
    valid_max:              40.0
    valid_min:              -2.0

In [16]:
psize = psc(chlarr,sstregrid)
psize

<xarray.Dataset> Size: 9MB
Dimensions:    (time: 2, lat: 300, lon: 350)
Coordinates:
    SST        (time, lat, lon) float64 2MB 21.39 21.39 21.39 ... 21.39 21.39
  * time       (time) datetime64[ns] 16B 2024-07-19 2024-07-27
    latitude   (lat) float32 1kB 49.95 49.85 49.75 49.65 ... 20.25 20.15 20.05
    longitude  (lon) float32 1kB -79.95 -79.85 -79.75 ... -45.25 -45.15 -45.05
  * lat        (lat) float32 1kB 49.95 49.85 49.75 49.65 ... 20.25 20.15 20.05
  * lon        (lon) float32 1kB -79.95 -79.85 -79.75 ... -45.25 -45.15 -45.05
Data variables:
    fpico      (time, lat, lon) float64 2MB nan nan nan ... 0.4505 0.4496 0.4484
    fnanopico  (time, lat, lon) float64 2MB nan nan nan ... 0.8007 0.8003 0.7999
    fnano      (time, lat, lon) float64 2MB nan nan nan ... 0.3502 0.3507 0.3515
    fmicro     (time, lat, lon) float64 2MB nan nan nan ... 0.1993 0.1997 0.2001
    chlor_a    (time, lat, lon) float32 840kB nan nan nan ... 0.04728 0.04951

In [17]:
psize.to_netcdf("psize_output.nc")

### Find SST values below/above the min/max sst in the LUT and change to the min/max values

In [21]:
sstregrid = xr.where(sstregrid>np.max(sst_table[:,0]),np.max(sst_table[:,0]), sstregrid)
sstregrid = xr.where(sstregrid<np.min(sst_table[:,0]),np.min(sst_table[:,0]), sstregrid)

NameError: name 'sst_table' is not defined

### Merge CHL and SST into a single dataset

In [22]:
chlsst = xr.merge([chlarr, sstregrid])
chlsst

<xarray.Dataset> Size: 2MB
Dimensions:                  (time: 2, lat: 300, lon: 350, rgb: 3,
                              eightbitcolor: 256)
Coordinates:
  * lat                      (lat) float32 1kB 49.95 49.85 49.75 ... 20.15 20.05
  * lon                      (lon) float32 1kB -79.95 -79.85 ... -45.15 -45.05
  * time                     (time) datetime64[ns] 16B 2024-07-19 2024-07-27
    latitude                 (lat) float32 1kB 49.95 49.85 49.75 ... 20.15 20.05
    longitude                (lon) float32 1kB -79.95 -79.85 ... -45.15 -45.05
Dimensions without coordinates: rgb, eightbitcolor
Data variables:
    chlor_a                  (time, lat, lon) float32 840kB nan nan ... 0.04951
    palette                  (time, rgb, eightbitcolor) uint8 2kB 147 0 ... 0 0
    sea_surface_temperature  (time, lat, lon) float32 840kB nan nan ... 27.55

In [52]:
chlsst

<xarray.Dataset> Size: 3MB
Dimensions:                  (lat: 300, lon: 350, time: 4, rgb: 3,
                              eightbitcolor: 256)
Coordinates:
  * lat                      (lat) float32 1kB 49.95 49.85 49.75 ... 20.15 20.05
  * lon                      (lon) float32 1kB -79.95 -79.85 ... -45.15 -45.05
  * time                     (time) object 32B 2024-07-19T00:18:26+00:00 ... ...
    latitude                 (lat) float32 1kB 49.95 49.85 49.75 ... 20.15 20.05
    longitude                (lon) float32 1kB -79.95 -79.85 ... -45.15 -45.05
Dimensions without coordinates: rgb, eightbitcolor
Data variables:
    chlor_a                  (time, lat, lon) float32 2MB nan nan ... nan nan
    palette                  (time, rgb, eightbitcolor) float32 12kB 147.0 .....
    sea_surface_temperature  (time, lat, lon) float32 2MB nan nan ... 21.39

In [50]:
xr.apply_ufunc(
    psc,
    chlsst.chlor_a,
    chlsst.sea_surface_temperature,
    input_core_dims=[["time", "lat", "lon"], ["time", "lat", "lon"]],
    vectorize=True,
)

ValueError: operands could not be broadcast together with shapes (293,) (4,300,350) 

In [43]:
sst_coeffs = sst_table[np.argmin(np.abs(sst_table[:, 0] - sst))]
sst_coeffs

array([9.999, 0.763, 0.784, 0.224, 0.353])

In [38]:
sstlut = pandas.read_csv(sst_file)
sst_table = np.array(sstlut)
minsst = np.min(sst_table[:,0])
maxsst = np.max(sst_table[:,0])

sst_arr = 10

In [19]:
sstregrid

<xarray.DataArray 'sea_surface_temperature' (time: 2, lat: 300, lon: 350)> Size: 840kB
array([[[      nan,       nan,       nan, ..., 14.729999, 15.276001,
         15.324   ],
        [      nan,       nan,       nan, ..., 13.902   , 14.606   ,
         15.256   ],
        [      nan,       nan,       nan, ..., 13.851999, 13.894   ,
         14.466001],
        ...,
        [29.649998, 29.694286, 29.715715, ..., 27.625713, 27.637142,
         27.637142],
        [29.6825  , 29.708572, 29.777143, ..., 27.60857 , 27.625717,
         27.63    ],
        [29.741251, 29.712858, 29.78143 , ..., 27.720001, 27.630001,
         27.57125 ]],

       [[      nan,       nan,       nan, ..., 16.886665, 16.706   ,
         17.101667],
        [      nan,       nan,       nan, ..., 16.406002, 16.816   ,
         16.832   ],
        [      nan,       nan,       nan, ..., 15.858   , 16.028   ,
         16.395   ],
        ...,
        [29.713331, 29.794   , 29.748001, ..., 27.843748, 27.692858,
         27.56286 ],
        [29.720001, 29.75    , 29.835714, ..., 27.830002, 27.77857 ,
         27.582   ],
        [29.700003, 29.713333, 29.807142, ..., 27.757143, 27.57429 ,
         27.552856]]], dtype=float32)
Coordinates:
  * time       (time) datetime64[ns] 16B 2024-07-19 2024-07-27
    latitude   (lat) float32 1kB 49.95 49.85 49.75 49.65 ... 20.25 20.15 20.05
    longitude  (lon) float32 1kB -79.95 -79.85 -79.75 ... -45.25 -45.15 -45.05
  * lat        (lat) float32 1kB 49.95 49.85 49.75 49.65 ... 20.25 20.15 20.05
  * lon        (lon) float32 1kB -79.95 -79.85 -79.75 ... -45.25 -45.15 -45.05
Attributes:
    colorBarMaximum:        35.0
    colorBarMinimum:        0.0
    comment:                SST obtained by regression with buoy measurements...
    coverage_content_type:  physicalMeasurement
    ioos_category:          Temperature
    long_name:              sea surface sub-skin temperature
    source:                 NOAA
    standard_name:          sea_surface_subskin_temperature
    units:                  degree_C
    valid_max:              40.0
    valid_min:              -2.0

In [20]:
sstregrid = xr.where(sstregrid>maxsst,maxsst, sstregrid)
sstregrid = xr.where(sstregrid<minsst,minsst, sstregrid)

<xarray.DataArray 'sea_surface_temperature' (time: 2, lat: 300, lon: 350)> Size: 840kB
array([[[      nan,       nan,       nan, ..., 14.729999, 15.276001,
         15.324   ],
        [      nan,       nan,       nan, ..., 13.902   , 14.606   ,
         15.256   ],
        [      nan,       nan,       nan, ..., 13.851999, 13.894   ,
         14.466001],
        ...,
        [21.388   , 21.388   , 21.388   , ..., 21.388   , 21.388   ,
         21.388   ],
        [21.388   , 21.388   , 21.388   , ..., 21.388   , 21.388   ,
         21.388   ],
        [21.388   , 21.388   , 21.388   , ..., 21.388   , 21.388   ,
         21.388   ]],

       [[      nan,       nan,       nan, ..., 16.886665, 16.706   ,
         17.101667],
        [      nan,       nan,       nan, ..., 16.406002, 16.816   ,
         16.832   ],
        [      nan,       nan,       nan, ..., 15.858   , 16.028   ,
         16.395   ],
        ...,
        [21.388   , 21.388   , 21.388   , ..., 21.388   , 21.388   ,
         21.388   ],
        [21.388   , 21.388   , 21.388   , ..., 21.388   , 21.388   ,
         21.388   ],
        [21.388   , 21.388   , 21.388   , ..., 21.388   , 21.388   ,
         21.388   ]]], dtype=float32)
Coordinates:
  * time       (time) datetime64[ns] 16B 2024-07-19 2024-07-27
    latitude   (lat) float32 1kB 49.95 49.85 49.75 49.65 ... 20.25 20.15 20.05
    longitude  (lon) float32 1kB -79.95 -79.85 -79.75 ... -45.25 -45.15 -45.05
  * lat        (lat) float32 1kB 49.95 49.85 49.75 49.65 ... 20.25 20.15 20.05
  * lon        (lon) float32 1kB -79.95 -79.85 -79.75 ... -45.25 -45.15 -45.05

In [34]:
fpico = (sst_coeffs.COEFF3 * (1 - exp(-1 * (sst_coeffs[4] / sst_coeffs[3]) * chl))) / chl
fnanopico = (sst_coeffs[1] * (1 - exp(-1 * (sst_coeffs[2] / sst_coeffs[1]) * chl))) / chl  
fnano = fnanopico - fpico
fmicro = (chl - (sst_coeffs[1] * (1 - exp(-1 * (sst_coeffs[2] / sst_coeffs[1]) * chl)))) / chl           

In [32]:
sst_arr[(sst_arr[:, 0] > 10) * (sst_arr[:, 0] < 15)]

NameError: name 'sst_arr' is not defined

In [21]:
sst_table[(sst_arr[:, 0] > 10) * (sst_arr[:, 0] < 15)]

,SST,COEFF1,COEFF2,COEFF3,COEFF4
60,10.050,0.767,0.788,0.227,0.353
61,10.100,0.767,0.788,0.228,0.352
62,10.149,0.757,0.788,0.226,0.353
63,10.199,0.751,0.791,0.226,0.354
64,10.250,0.743,0.793,0.225,0.353
...,...,...,...,...,...
174,14.827,1.050,0.794,0.348,0.254
175,14.864,1.048,0.801,0.350,0.256
176,14.899,1.052,0.804,0.353,0.257
177,14.936,1.061,0.805,0.356,0.259
